# Load libraries & helper functions

In [ ]:
from fmriprepciftify_helpers import *
%matplotlib inline

# Set parameters

In [49]:
# config is a global variable used by several functions
# Where does the data live?
config.DATADIR = '/data2/jdubois2/data/YALE-trt/derivatives'
config.sourceDir = os.getcwd() # or replace with path to source code
config.outDir             = 'rsDenoise' # where all output of processing will be, in BIDS like format
config.FCDir              = 'FC' # where all the time series used to compute FC and final FC matrices will be stored

# Which resting-state denoising pipeline should be used?
config.pipelineName            = 'A'
config.Operations              = config.operationDict[config.pipelineName]

# Other options
config.isCifti = False
config.queue = False
config.sgeopts = '-l mem_free=25G -pe openmp 6 -q long.q'
config.parcellationName = 'shen2013'
config.parcellationFile = '/data/jdubois/data/parcellations/shenetal_neuroimage2013_new/shen_2mm_268_parcellation.nii.gz'
config.nParcels = 268
config.overwrite = False
config.Operations = config.operationDict[config.pipelineName]


sessions = ['ses-001TB', 'ses-002TA', 'ses-003TB', 'ses-004TA',
            'ses-001TA', 'ses-002TB', 'ses-003TA', 'ses-004TB', ]
fmriRuns = ['task-rest_run-01','task-rest_run-02','task-rest_run-03',
            'task-rest_run-04','task-rest_run-05','task-rest_run-06']
subjects = ['sub-032401', 'sub-032403', 'sub-032405', 'sub-032407', 'sub-032411',
            'sub-032402', 'sub-032404', 'sub-032408', 'sub-032410', 'sub-032412']

# which file to use for the functional data?
#the code #fMRIrun# will be replaced by the appropriate run
config.fmriFileTemplate        = '#fMRIrun#_Atlas_MSMAll.dtseries.nii'
## do not alter the following lines ##
##>>>>>>>>
tmp = config.fmriFileTemplate.split('.')
if tmp[1]=='nii':
    config.isCifti = False
elif tmp[1]=='dtseries':
    config.isCifti = True
else:
    print 'unknown file extension'
##<<<<<<<<
# parcellation for FC matrix
config.parcellationName        = 'Glasser' #used for easy reference
config.parcellationFile        = '/scratch/duboisjx/data/parcellations/Glasser2016/Parcels.dlabel.nii'
config.nParcels                = 360
# where are the .csv files with subject scores and info?
# unrestricted
config.behavFile               = '/scratch/duboisjx/data/HCP/neuropsych/unrestricted_luckydjuju_6_7_2017_0_44_41.csv'
# RESTRICTED: needed for age, handedness, family structure,...
config.RbehavFile              = '/scratch/duboisjx/data/HCP/neuropsych/RESTRICTED_luckydjuju_8_22_2017_11_13_18.csv'
# other naming conventions
config.melodicFolder           = op.join('#fMRIrun#_hp2000.ica','filtered_func_data.ica') 
config.movementRelativeRMSFile = 'Movement_RelativeRMS.txt'
config.movementRegressorsFile  = 'Movement_Regressors_dt.txt'
# it is advisable to run the analyses on a cluster with sge
config.queue        = True
parallelEnvironment = 'smp' #'openmp'
# output directory
outDir              = op.join(config.DATADIR,'Results','INTELLIGENCE',config.pipelineName,config.parcellationName)
if not op.isdir(outDir):
    makedirs(outDir)
    
# if working with volumetric data: should the parcels be restricted to the gray matter mask?
if not config.isCifti:
    config.maskParcelswithGM       = False
    if config.maskParcelswithGM:
        config.parcellationName = config.parcellationName + '_GM'

### MRI processing

In [ ]:
for config.subject in subjects:
    for config.session in sessions:
        for config.fmriRun in fmriRuns:
            if runPipelinePar():
                computeFC(overwrite=True)

### Retrieve FC

In [ ]:
## 
#  @brief Get FC matrices for list of subjects
#  
#  @param [array-like] subjectList list of subject IDs
#  @param [array-like] runs list of runs 
#  @param [array-like] sessions list of sessions (optional)
#  @param [str] parcellation parcellation name - needed if FCDir is None      
#  @param [list] operations pipeline operations - needed if FCDir is None
#  @param [str] outputDir path to preprocessed data folder (optional, default is outpath())
#  @param [bool] isCifti True if preprocessed data is in cifti format
#  @param [str] fcMatFile full path to output file (default ./fcMats.mat)
#  @param [str] kind type of FC, one of {"correlation" (default), "partial correlation", "tangent", "covariance", "precision"}
#  @param [bool] overwrite True if existing files should be overwritten
#  @param [str] FCDir path to folder containing precomputed timeseries x parcels per subject - if None they are retrieved from each subject's folder
#  @param [bool] mergeSessions True if time series from different sessions should be merged before computing FC, otherwise FC from each session are averaged
#  @param [bool] mergeRuns True if time series from different runs should be merged before computing FC, otherwise FC from each run are averaged (if mergeSessions is True mergeRuns is ignored and everything is concatenated)
#  @param [CovarianceEstimator] cov_estimator is None, default sklearn.covariance.LedoitWolf estimator is used

results = getAllFC(subjects,fmriRuns,sessions,config.parcellationName,config.Operations,mergeSessions=False,mergeRuns=False, 
                   FCDir=config.FCDir)